# Import modules

In [8]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import copy
import time
from collections import Counter
from fuzzywuzzy import fuzz

In [33]:
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import commonplayerinfo, playercareerstats, commonallplayers

# Team Data

In [10]:
data_teams = pd.DataFrame(teams.get_teams())
data_teams.head()

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


In [150]:
data_teams.to_csv('teams_data.csv', index = False)

### Team Salaries

In [11]:
years = {'2021/22': '', '2022/23': '2022', '2023/24': '2023', '2024/25': '2024'}

for y in years.keys():
    
    url = 'https://www.spotrac.com/nba/cap/' + years[y]
    r = requests.get(url, timeout=2.5)
    r_html = r.text
    soup = BeautifulSoup(r_html, 'html.parser')

    salary_table = soup.find('table')
    
    if y == '2021/22':
        cap_space = [[salary_table.find_all("td")[i].text.strip() for i in [(j-1)*12+1, (j-1)*12+9]] for j in range(1, 31)]
        salary_cap_teams = pd.DataFrame(cap_space, columns=['Team', y])
        print(y)
        continue
    else:
        cap_space = [[salary_table.find_all("td")[i].text.strip() for i in [(j-1)*11+1, (j-1)*11+8]] for j in range(1, 31)]
        print(y)
    
    
    salary_cap_year = pd.DataFrame(cap_space, columns=['Team', y])
    
    salary_cap_teams = salary_cap_teams.merge(salary_cap_year, on = 'Team', how = 'left')

2021/22
2022/23
2023/24
2024/25


In [12]:
salary_cap_teams.insert(1, 'Abb', salary_cap_teams['Team'].str[-3:])
salary_cap_teams['Team'] = salary_cap_teams['Team'].str[:-3]
salary_cap_teams.insert(0, 'Team_ID', [list(data_teams[data_teams['full_name'] == salary_cap_teams['Team'][i]]['id'])[0] for i in range(salary_cap_teams.shape[0])])

# lakers in 2021: -50 mil -> 50 mil over Cap

In [13]:
for col in salary_cap_teams.columns[3:]:
    salary_cap_teams[col] = salary_cap_teams[col].apply(lambda x: x.replace('$', '').replace(',', '')).astype(float)
        
salary_cap_teams

,Team_ID,Team,Abb,2021/22,2022/23,2023/24,2024/25
0,1610612760,Oklahoma City Thunder,OKC,24572406.0,-1848998.0,-9418667.0,41399666.0
1,1610612763,Memphis Grizzlies,MEM,-3347775.0,-15442836.0,-30850268.0,57216487.0
2,1610612740,New Orleans Pelicans,NOP,-4004104.0,-15614516.0,-45768872.0,-7268819.0
3,1610612766,Charlotte Hornets,CHA,-4490105.0,-18232263.0,-20502290.0,8171964.0
4,1610612752,New York Knicks,NYK,-5042436.0,-18174322.0,-35439037.0,-22895991.0
5,1610612759,San Antonio Spurs,SAS,-5498958.0,-5657182.0,6625794.0,23360149.0
6,1610612765,Detroit Pistons,DET,-12756440.0,19834484.0,8496824.0,34931858.0
7,1610612756,Phoenix Suns,PHX,-15959873.0,-54333058.0,-53905519.0,5154000.0
8,1610612745,Houston Rockets,HOU,-17647614.0,-21705994.0,-37760728.0,44214930.0
9,1610612753,Orlando Magic,ORL,-18026594.0,-49345552.0,-8848482.0,-22092877.0


In [149]:
salary_cap_teams.to_csv('teams_salaries.csv', index = False)

# Players

### Player salaries

In [25]:
url='https://hoopshype.com/salaries/players/'

r = requests.get(url, timeout=2.5)
r_html = r.text

soup = BeautifulSoup(r_html, 'html.parser')

salary_table = soup.find('table')

In [26]:
length=len(salary_table.find_all("td"))

player_names=[salary_table.find_all("td")[i].text.strip() for i in range(9,length,8)]

column1=[salary_table.find_all("td")[i].text.strip() for i in range(10,length,8)]
column2=[salary_table.find_all("td")[i].text.strip() for i in range(11,length,8)]
column3=[salary_table.find_all("td")[i].text.strip() for i in range(12,length,8)]
column4=[salary_table.find_all("td")[i].text.strip() for i in range(13,length,8)]
column5=[salary_table.find_all("td")[i].text.strip() for i in range(14,length,8)]
column6=[salary_table.find_all("td")[i].text.strip() for i in range(15,length,8)]

In [27]:
dict_df = {'player_names':player_names,
         # '2019/20':column1,
         # '2020/21':column2,
          '2021/22':column3,
          '2022/23':column4,
          '2023/24':column5,
          '20124/25':column6}
        
player_salary = pd.DataFrame(dict_df)

In [28]:
player_salary

,player_names,2021/22,2022/23,2023/24,20124/25
0,Stephen Curry,"$51,915,615","$55,761,217","$59,606,817",$0
1,James Harden,$0,$0,$0,$0
2,John Wall,$0,$0,$0,$0
3,Russell Westbrook,$0,$0,$0,$0
4,LeBron James,$0,$0,$0,$0
...,...,...,...,...,...
531,Admiral Schofield,$0,$0,$0,$0
532,Troy Williams,$0,$0,$0,$0
533,Mamadi Diakite,$0,$0,$0,$0
534,Demetrius Jackson,"$92,857",$0,$0,$0


In [29]:
player_salary = player_salary.rename(columns={"20124/25": "2024/25"})

In [30]:
salary_years = ['2021/22', '2022/23', '2023/24', '2024/25']

In [31]:
for col in salary_years:
    player_salary[col] = player_salary[col].apply(lambda x: x.replace('$', '').replace(',', '')).astype(float)
    
    
# + also discount as in scratchpad.ipynb to get NPV?

## Match players from NBA API to Salaries -> get Player-IDs

In [51]:
response = commonallplayers.CommonAllPlayers(is_only_current_season = ['0'],season=['2021']).get_data_frames()[0] # get all players & indicate current season for rosterstatus
response_new = response[(response['ROSTERSTATUS'] == 1) # get active players -> players who retired at end of last season are not contained (e.g. 'JJ Redick')
                        & ((response['FROM_YEAR'] != '2021')) # no current rookies
                       ].reset_index()
print(response_new.shape)

(418, 17)


In [52]:
# match players from salary table and from nba table to get unique IDs

player_ids = []
player_names = [list(response_new['DISPLAY_FIRST_LAST'])[j] for j in range(response_new.shape[0])]
player_names.append('unk') # such that list 
for i in range(player_salary.shape[0]):
    player_name = player_salary['player_names'][i]
    try:
        player_ids.append(list(response_new[response_new['DISPLAY_FIRST_LAST'] == player_name]['PERSON_ID'])[0])
        try:
            player_names.remove(player_name)
        except ValueError:
            print('Here', player_name)
        
    except IndexError:
        player_ids.append(0)

for i in range(len(player_ids)):
    if player_ids[i] == 0:
        player_name = player_salary['player_names'][i]
        fuzz_scores = np.array([fuzz.ratio(player_name, player_names[j]) for j in range(len(player_names))])
        max_ind = np.argmax(fuzz_scores)
        if fuzz_scores[max_ind] > 65:
            matched_player_name = player_names[max_ind] #list(response_new['DISPLAY_FIRST_LAST'])[max_ind]
            print(f"{player_name} matched to {matched_player_name}")
            player_ids[i] = list(response_new[response_new['DISPLAY_FIRST_LAST'] == matched_player_name]['PERSON_ID'])[0]
            player_names.remove(matched_player_name)
        else:
            #player_ids.append(0)
            print(f"{player_name} NOT matched to {player_names[max_ind]}")

print(player_names)

Tim Hardaway Jr matched to Tim Hardaway Jr.
Gary Trent Jr matched to Gary Trent Jr.
Marcus Morris matched to Marcus Morris Sr.
TJ Warren matched to T.J. Warren
Kelly Oubre matched to Kelly Oubre Jr.
Marvin Bagley matched to Marvin Bagley III
Devonte Graham matched to Devonte' Graham
Larry Nance Jr matched to Larry Nance Jr.
Al-Farouq Aminu NOT matched to Nic Claxton
Cade Cunningham NOT matched to Juancho Hernangomez
Derrick Jones matched to Derrick Jones Jr.
Jaren Jackson Jr matched to Jaren Jackson Jr.
Jalen Green NOT matched to Juancho Hernangomez
Evan Mobley NOT matched to Danuel House Jr.
TJ McConnell matched to T.J. McConnell
Scottie Barnes NOT matched to Maurice Harkless
PJ Tucker matched to P.J. Tucker
DJ Augustin matched to D.J. Augustin
Wendell Carter matched to Wendell Carter Jr.
Juan Hernangomez matched to Juancho Hernangomez
Jalen Suggs NOT matched to Danuel House Jr.
Joakim Noah NOT matched to R.J. Hampton
Josh Giddey NOT matched to Ish Smith
Patrick Mills matched to Patty

In [68]:
further_player_ids = []
for i in range(len(player_names)-1):
    player_name = player_names[i]
    fuzz_scores = np.array([fuzz.ratio(player_name, list(player_salary['player_names'])[j]) for j in range(player_salary.shape[0])])
    max_ind = np.argpartition(fuzz_scores, -3)[-3:]
    possbile_matches = player_salary['player_names'][max_ind].reset_index(drop = True) #list(response_new['DISPLAY_FIRST_LAST'])[max_ind]
    print(f"{player_name} could be matched to the following 3 players\n{possbile_matches}")
    matched_id = int(input('Which player should be chosen (please indicate the index)?\n'))
    further_player_ids.append(possbile_matches[matched_id])
    print(f"Added {possbile_matches[matched_id]} for {player_name}")
    
    # Kenyon Martin Jr. -> KJ Martin

Kenyon Martin Jr. could be matched to the following 3 players
0     Cody Martin
1       KJ Martin
2    Kelan Martin
Name: player_names, dtype: object
Which player should be chosen (please indicate the index)?
1
Added KJ Martin for Kenyon Martin Jr.


In [88]:
for i in range(len(further_player_ids)):
    idx_matched_player = player_salary.index[player_salary['player_names'] == further_player_ids[0]].to_list()[0]
    player_ids[idx_matched_player] = list(response_new[response_new['DISPLAY_FIRST_LAST'] == player_names[i]]['PERSON_ID'])[0]
    
# player_ids[428] = list(response_new[response_new['DISPLAY_FIRST_LAST'] == player_names[0]]['PERSON_ID'])[0] # Kenyon Martin Jr. -> KJ Martin

In [89]:
player_salary.insert(0, "id", player_ids)

In [90]:
# players which could not be matched, but have salary in current seasons: rookies

player_salary[(player_salary['id'] == 0) & (player_salary['2021/22'] != 0)]

,id,player_names,2021/22,2022/23,2023/24,2024/25
126,0,Cade Cunningham,11055360.0,13940809.0,18123052.0,0.0
143,0,Jalen Green,9891480.0,12483048.0,16290377.0,0.0
162,0,Evan Mobley,8882640.0,11227657.0,14730686.0,0.0
170,0,Scottie Barnes,8008680.0,10130980.0,13362763.0,0.0
179,0,Jalen Suggs,7252080.0,9188385.0,12183799.0,0.0
187,0,Josh Giddey,6587040.0,8352367.0,11142057.0,0.0
197,0,Jonathan Kuminga,6012840.0,7636307.0,10240287.0,0.0
209,0,Franz Wagner,5508720.0,7007092.0,9445560.0,0.0
227,0,Davion Mitchell,5063640.0,6451077.0,8741210.0,0.0
235,0,Ziaire Williams,4810200.0,6133005.0,8353152.0,0.0


In [91]:
player_salaries = player_salary[player_salary['id'] != 0].reset_index(drop = True) # exclude rookies
player_salaries.head()

,id,player_names,2021/22,2022/23,2023/24,2024/25
0,201939,Stephen Curry,51915615.0,55761217.0,59606817.0,0.0
1,201935,James Harden,0.0,0.0,0.0,0.0
2,202322,John Wall,0.0,0.0,0.0,0.0
3,201566,Russell Westbrook,0.0,0.0,0.0,0.0
4,2544,LeBron James,0.0,0.0,0.0,0.0


In [92]:
f = dict(Counter(player_salaries['id']))
[k for k,v in f.items() if (v > 1) | (k == 0)] # -> unique keys and only the ones with valid ID

[]

In [93]:
player_salary[player_salary['player_names'] == 'Klay Thompson']

,id,player_names,2021/22,2022/23,2023/24,2024/25
10,202691,Klay Thompson,43219440.0,0.0,0.0,0.0


In [148]:
player_salaries.to_csv('players_salaries.csv', index = False)

### Notes

#### Approach:
- Compare with list: current active players (have contract) - rookies -> für jeden active player: ID

#### Alternative:
- Basis alle Spieler, der letzten 4 Saisons
- Contract player ids LEFT JOIN ids active past 4 seasons

## Player Table & Stats

In [136]:
players = copy.deepcopy(player_salaries.iloc[: , :-4])
display(players.head())
print(players.shape)

,id,player_names
0,201939,Stephen Curry
1,201935,James Harden
2,202322,John Wall
3,201566,Russell Westbrook
4,2544,LeBron James


(418, 2)


In [95]:
stats = []
for player in tqdm(players['id']):
    time.sleep(.600)
    call_career = playercareerstats.PlayerCareerStats(player_id=player) 
    stats.append(call_career.get_data_frames()[0])
    
stats_players = pd.concat(stats)
stats_players.head()

100%|██████████| 418/418 [14:57<00:00,  2.15s/it]


,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,201939,2009-10,00,1610612744,GSW,22.0,80,77,2896.0,528,...,0.885,48,308,356,472,152,19,243,252,1399
1,201939,2010-11,00,1610612744,GSW,23.0,74,74,2489.0,505,...,0.934,52,234,286,432,109,20,226,233,1373
2,201939,2011-12,00,1610612744,GSW,24.0,26,23,732.0,145,...,0.809,15,73,88,138,39,8,65,62,383
3,201939,2012-13,00,1610612744,GSW,25.0,78,78,2983.0,626,...,0.900,59,255,314,539,126,12,240,198,1786
4,201939,2013-14,00,1610612744,GSW,26.0,78,78,2846.0,652,...,0.885,46,288,334,666,128,14,294,194,1873


In [96]:
stats_players.to_csv('playercareerstats.csv', index = False)

In [109]:
stats_players = pd.read_csv('playercareerstats.csv')

In [110]:
stats_players = stats_players[(stats_players['SEASON_ID'] == '2020-21') | (stats_players['SEASON_ID'] == '2019-20') | (stats_players['SEASON_ID'] == '2018-19')].reset_index().drop(columns=['index'])

In [111]:
print(stats_players.columns)

Index(['PLAYER_ID', 'SEASON_ID', 'LEAGUE_ID', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'PLAYER_AGE', 'GP', 'GS', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')


In [112]:
def combine_seasons(player_id):
    df = stats_players[stats_players['PLAYER_ID'] == player_id]
    
    season_20 = df[df['SEASON_ID'] == '2020-21']
    if season_20.shape[0] == 0:
        season_20 = pd.DataFrame(np.zeros((1, len(df.columns) -6)))
    elif season_20.shape[0] > 1:
        season_20 = season_20[season_20['TEAM_ABBREVIATION'] == 'TOT'].iloc[:,6:] * 1/2
    else:
        season_20 = season_20.iloc[:,6:] * 1/2
   # except:
   #     season_20 = pd.DataFrame(np.zeros((1, len(df.columns) -6)))
    
    season_19 = df[df['SEASON_ID'] == '2019-20']
    if season_19.shape[0] == 0:
        season_19 = pd.DataFrame(np.zeros((1, len(df.columns) -6)))
    elif season_19.shape[0] > 1:
        season_19 = season_19[season_19['TEAM_ABBREVIATION'] == 'TOT'].iloc[:,6:] * 2/6
    else:
        season_19 = season_19.iloc[:,6:] * 2/6
    
    #season_19 = pd.DataFrame(np.zeros((1, len(df.columns) -6)))
    
    season_18 = df[df['SEASON_ID'] == '2018-19']
    if season_18.shape[0]  == 0:
        season_18 = pd.DataFrame(np.zeros((1, len(df.columns) -6)))
    elif season_18.shape[0] > 1:
        season_18 = season_18[season_18['TEAM_ABBREVIATION'] == 'TOT'].iloc[:,6:] * 1/6
    else:
        season_18 = season_18.iloc[:,6:] * 1/6
    #except:
    #    season_18 = pd.DataFrame(np.zeros((1, len(df.columns) -6)))
    
    values_pastSeasons = (season_20.values + season_19.values + season_18.values).flatten()
    
    if sum(values_pastSeasons) == 0:
        player_name = list(players[players['id'] == player_id]['player_names'])[0]
        print(f'No game data: {player_name} with id {player_id}')
        return 'NA'
    
    df_final = copy.deepcopy(df)
    df_final.iloc[0, 6:] = values_pastSeasons

    df_final.iloc[0, 1] = 'aggregated'
    #df_final.iloc[0, 4] = 'NA'
     
    dict_final = dict(df_final.iloc[0])
    return dict_final #pd.DataFrame.from_dict(list(r.values()), orient='index', columns = list(r.keys()))

# combine_seasons(201935)

In [137]:
stats_players_agg = [combine_seasons(player_id) for player_id in players['id']]
ind_player_drop = stats_players_agg.index('NA')
stats_players_agg = [x for x in stats_players_agg if x != 'NA']
stats_players_agg = pd.DataFrame(stats_players_agg)
stats_players_agg

No game data: Omer Yurtseven with id 1630209


,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,201939,aggregated,0,1610612744,GSW,31.0,44.666667,44.666667,1510.833333,445.333333,...,0.944000,23.333333,219.333333,242.666667,252.666667,55.500000,8.833333,143.833333,90.833333,1355.666667
1,201935,aggregated,0,1610612745,HOU,29.0,57.666667,57.166667,2110.000000,535.500000,...,0.865333,51.833333,357.166667,409.000000,505.833333,94.500000,46.166667,255.666667,165.833333,1789.500000
2,202322,aggregated,0,1610612764,WAS,28.0,25.333333,25.333333,828.000000,187.333333,...,0.490667,11.000000,72.833333,83.833333,184.000000,29.166667,20.333333,90.666667,34.833333,522.000000
3,201566,aggregated,0,1610612760,OKC,30.0,63.666667,63.666667,2305.666667,578.333333,...,0.691667,106.000000,553.833333,659.833333,645.833333,99.166667,23.666667,295.166667,202.500000,1519.333333
4,2544,aggregated,0,1610612747,LAL,34.0,54.000000,54.000000,1846.833333,518.333333,...,0.690833,46.000000,379.500000,425.500000,478.666667,62.000000,30.000000,203.833333,90.000000,1379.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,1629719,aggregated,0,1610612747,LAL,23.0,10.333333,0.500000,52.000000,9.500000,...,0.227500,6.666667,11.000000,17.666667,0.833333,2.500000,2.000000,2.500000,4.000000,21.500000
413,1630238,aggregated,0,1610612746,LAC,23.0,1.500000,0.000000,5.500000,0.500000,...,0.000000,0.000000,1.500000,1.500000,0.000000,0.000000,0.000000,0.000000,1.500000,1.500000
414,1630235,aggregated,0,1610612762,UTA,23.0,15.000000,0.000000,151.000000,16.000000,...,0.500000,5.500000,16.500000,22.000000,22.500000,4.500000,2.000000,10.500000,11.000000,44.000000
415,1630216,aggregated,0,1610612764,WAS,23.0,11.000000,0.000000,49.000000,7.000000,...,0.416500,0.000000,4.500000,4.500000,6.000000,1.000000,0.500000,2.500000,4.000000,20.500000


In [151]:
stats_players_agg.to_csv('playercareerstats_agg.csv', index = False)

In [139]:
# update players_list
print(players.shape)
#players = players[players['id'] != 1630209].reset_index(drop = True)
players = players.drop(ind_player_drop).reset_index(drop = True)
print(players.shape)

(418, 2)
(417, 2)


### Add position

In [141]:
def adj_position(pos):
    if pos[0] in ['C', 'F', 'G']:
        return pos[0]
    else:
        print('Please enter a valid position.')
        pass

In [142]:
# add positions of players
positions_raw = []
for p_id in tqdm(players['id']):
    time.sleep(.600)
    positions_raw.append(commonplayerinfo.CommonPlayerInfo(p_id).get_data_frames()[0]['POSITION'][0])
print(len(positions_raw))

positions = [adj_position(position) for position in positions_raw]
Counter(positions)

100%|██████████| 417/417 [07:22<00:00,  1.06s/it]

417


Counter({'G': 200, 'F': 165, 'C': 52})

In [143]:
players['position'] = positions

In [144]:
players.head()

,id,player_names,position
0,201939,Stephen Curry,G
1,201935,James Harden,G
2,202322,John Wall,G
3,201566,Russell Westbrook,G
4,2544,LeBron James,F


### Add team from end of last season they played

In [145]:
teams_players = []
for i in range(len(players)):
    id_player = players['id'][i]
   
    teams = list(stats_players[(stats_players['PLAYER_ID'] == id_player) & (stats_players['SEASON_ID'] == '2020-21')]['TEAM_ABBREVIATION'])
    try:
        if teams[-1] == 'TOT':
            teams_players.append(teams[-2])
        else:
            teams_players.append(teams[-1])
            
    except IndexError:
        #print(players['player_names'][i], players['id'][i])
        
        teams = list(stats_players[(stats_players['PLAYER_ID'] == id_player) & (stats_players['SEASON_ID'] == '2019-20')]['TEAM_ABBREVIATION'])
        try:
            if teams[-1] == 'TOT':
                teams_players.append(teams[-2])
            else:
                teams_players.append(teams[-1])
                
        except IndexError:
            teams = list(stats_players[(stats_players['PLAYER_ID'] == id_player) & (stats_players['SEASON_ID'] == '2018-19')]['TEAM_ABBREVIATION'])
            try:
                if teams[-1] == 'TOT':
                    teams_players.append(teams[-2])
                else:
                    teams_players.append(teams[-1])
                    
            except IndexError:
                print(f"No team in the last three seasons could be found for {players['player_names'][i]} with id {players['id'][i]}")
            
            
# letzes aktives team

In [146]:
players['team'] = teams_players
display(players.head())

,id,player_names,position,team
0,201939,Stephen Curry,G,GSW
1,201935,James Harden,G,BKN
2,202322,John Wall,G,HOU
3,201566,Russell Westbrook,G,WAS
4,2544,LeBron James,F,LAL


In [147]:
players.to_csv('players_data.csv', index = False)